In [1]:
import numpy as np
import scipy as sp

import scipy
from scipy.stats import multivariate_normal
from scipy       import optimize

import skimage

import matplotlib.pyplot as plt

import pylab

from glob import glob
import sys
import os

from tqdm import tqdm

import math

from glob import glob

import matplotlib.pyplot as plt

import itk
from itk import TubeTK as ttk

import av
import ffmpeg

import json

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *

In [5]:
def extrude_column(ovrimg, col, val):
    col = int(col)
    z_count = 0
    z_first = 0
    for z in range(ovrimg.shape[0]):
        pos = np.where(ovrimg[z,:,col]==val)
        if len(pos[0])>0:
            z_count += 1
            z_first += pos[0][0]
    if z_count > ovrimg.shape[0]//4:
        first = z_first // z_count
        for z in range(ovrimg.shape[0]):
            pos = np.where(ovrimg[z,:,col]==val)
            if len(pos[0])>0:
                first = pos[0][0]
            ovrimg[z,:,col]=0
            ovrimg[z,first:min(first+50,ovrimg.shape[1]),col]=val
    else:
        ovrimg[:,:,col]=0
    
def extrude_annotations(pathname):
    data_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.mha"))
    data_overlay_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.interpolated-overlay.mha"))
    
    num_overlays = len(data_overlay_paths)
    for i in range(num_overlays):
        ovrimg = itk.imread(data_overlay_paths[i], itk.UC)
        print("Processing", data_overlay_paths[i])
        
        # Eliminate rib annotation
        #ovrarr = itk.GetArrayFromImage(ovrimg)
        #ovrarr = np.where(ovrarr==2,0,ovrarr)
        
        nos = 0
        rib = 0
        sli = 0
        for c in range(ovrimg.shape[2]):
            nos = np.count_nonzero(ovrimg[:,:,c]==1)
            rib = np.count_nonzero(ovrimg[:,:,c]==2)
            sli = np.count_nonzero(ovrimg[:,:,c]==3)
            if nos>0 and rib==0 and sli==0:
                extrude_column(ovrimg,c,1)
            elif nos==0 and rib>0 and sli==0:
                extrude_column(ovrimg,c,2)  #cause class 2 (rib) to be erased
            elif nos==0 and rib==0 and sli>0:
                extrude_column(ovrimg,c,3)
            else:
                ovrimg[:,:,c]=0
                
        (filepath,filename) = os.path.split(data_overlay_paths[i])
        new_name = filepath+"/"+filename.split('.')[0]+'.columns-roi-overlay.mha'
        print("Writing", new_name)
        ovrimg = itk.GetImageFromArray(ovrimg.astype(np.ubyte))
        itk.imwrite(ovrimg, new_name, compression=True)

In [ ]:
#pathname = "../../Data/VFoldData/BAMC-PTXNoSliding"
pathname = "../../Data/VFoldData/BAMC-PTXSliding"
extrude_annotations(pathname)

Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/025ns_Image_262499828648_clean.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/025ns_Image_262499828648_clean.columns-roi-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/025ns_image_267456908021_clean.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/025ns_image_267456908021_clean.columns-roi-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/026ns_image_1083297968960_clean.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/026ns_image_1083297968960_clean.columns-roi-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/026ns_image_1087766719219_clean.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXNoSliding-Annotations-Linear/026ns_image_1087766719219_clean.columns-roi-overlay.mha
Processing .